<a href="https://colab.research.google.com/github/TirendazAcademy/Hands-on-Data-Science-with-GCP/blob/main/Random%20Forest%20with%20Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Random Forest with Bigquery

Hi guys 😀 This notebook shows how to build a random forest model and predict the values. Let's get started.

## Authentication

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Creating dataset

In [2]:
%%bigquery --project tirendaz-data-engineering df
SELECT limit_balance, education_level, age, default_payment_next_month
FROM `bigquery-public-data.ml_datasets.credit_card_default`

In [3]:
df.head()

,limit_balance,education_level,age,default_payment_next_month
0,50000.0,3,50.0,0
1,200000.0,3,45.0,0
2,130000.0,1,45.0,0
3,230000.0,2,41.0,1
4,50000.0,2,42.0,0


## Creating input and output sets

In [4]:
target_column = "default_payment_next_month"
labels = df[target_column]
features = df.drop(target_column, axis = 1)

## Splitting sets into train and test

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
  train_test_split(features, labels, test_size=0.3)

## Building the random forest model

In [6]:
from sklearn.ensemble import RandomForestClassifier
random_forest_classifier = RandomForestClassifier(n_estimators=100)
random_forest_classifier.fit(X_train,y_train)

RandomForestClassifier()

## Making prediction

In [7]:
from sklearn import metrics
y_pred=random_forest_classifier.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7191011235955056


## Saving the model

In [8]:
import joblib
model_name = "cc_default_rf_model.sav"
joblib.dump(random_forest_classifier, model_name)

['cc_default_rf_model.sav']

## Loading the model

In [9]:
loaded_model = joblib.load(model_name)

Predicting some values

In [10]:
%%bigquery --project tirendaz-data-engineering df_predict
SELECT limit_balance, education_level, age, 
FROM `bigquery-public-data.ml_datasets.credit_card_default`
LIMIT 10;

### Let's take a look at the test set

In [11]:
df_predict.head()

,limit_balance,education_level,age
0,50000.0,3,50.0
1,200000.0,3,45.0
2,130000.0,1,45.0
3,230000.0,2,41.0
4,50000.0,2,42.0


### Let's predict the test set with the random forest model

In [12]:
prediction=loaded_model.predict(df_predict)
prediction

array(['0', '0', '0', '1', '0', '0', '1', '0', '1', '0'], dtype=object)

### Let's get an example and predict it with the model

In [13]:
import json
import pandas as pd
loaded_model = joblib.load(model_name)
limit_balance = 1000
education_level = 1
age = 25
feature_json = json.dumps([[limit_balance, education_level, age]])
feature_list = json.loads(feature_json)
feature_df = pd.DataFrame(feature_list)
prediction=loaded_model.predict(feature_df)
print("Online Prediction :")
print(prediction)

Online Prediction :
['0']


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"


Thanks for reading. Follow me on [Twitter](https://twitter.com/TirendazAcademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [YouTube](https://www.youtube.com/channel/UCFU9Go20p01kC64w-tmFORw) | [Tiktok](https://www.tiktok.com/@tirendazacademy) | [Medium](https://tirendazacademy.medium.com) | [Reddit](https://www.reddit.com/user/TirendazAcademy) 